In [1]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.contrib.slim.nets import inception
slim = tf.contrib.slim

In [2]:
tf.flags.DEFINE_string(
    'master', '', 'The address of the TensorFlow master to use.'
)

tf.flags.DEFINE_string(
    'checkpoint_path', '', 'Path to checkpoint for inception network.')

tf.flags.DEFINE_string(
    'input_dir', '', 'Input directory with images.')

tf.flags.DEFINE_string(
    'output_dir', '', 'Output directory with images.')

tf.flags.DEFINE_integer(
    'batch_size', 5, 'How many images process at one time.')

FLAGS = tf.flags.FLAGS

In [3]:
def load_target_class(input_dir):
    """Loads target classes."""
    with tf.gfile.Open(os.path.join(input_dir, 'target_class.csv')) as f:
        return {row[0]: int(row[1]) for row in csv.reader(f) if len(row) >= 2}


def load_images(input_dir, batch_shape):
    """Read png images from input directory in batches.

    Args:
        input_dir: input directory
        batch_shape: shape of minibatch array, i.e. [batch_size, height, width, 3]

    Yields:
        filenames: list file names without path of each image
            Lenght of this list could be less than batch_size, in this case only
            first few images of the result are elements of the minibatch.
        images: array with all images from this batch
    """
    images = np.zeros(batch_shape)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    for filepath in tf.gfile.Glob(os.path.join(input_dir, '*.png')):
        with tf.gfile.Open(filepath) as f:
            image = imread(f, mode='RGB').astype(np.float) / 255.0
        # Images for inception classifier are normalized to be in [-1, 1] interval.
        images[idx, :, :, :] = image * 2.0 - 1.0
        filenames.append(os.path.basename(filepath))
        idx += 1
        if idx == batch_size:
            yield filenames, images
            filenames = []
            images = np.zeros(batch_shape)
            idx = 0
    if idx > 0:
        yield filenames, images

In [4]:
class PushinMatyshin:
    def __init__(self):
        self.graph = tf.Graph()
        self.image_shape = (FLAGS.batch_size, 299, 299, 3)
        self.num_classes = 1001
        
        with self.graph.as_default():
            self.fake_image_subst = tf.Variable(
                np.zeros(self.image_shape, dtype=np.float32), name='fake_image_subst'
            )
            self.fake_image = (tf.tanh(self.fake_image_subst) + 1) / 2
        
            with slim.arg_scope(inception.inception_v3_arg_scope()):
                self.fake_logits, self.inception_end_points = inception.inception_v3(
                    self.fake_image, num_classes=self.num_classes, is_training=False,
                )
        
            self.real_image = tf.placeholder(tf.float32, self.image_shape, name='real_image')
            self.target_probs = tf.placeholder(
                tf.float32, [FLAGS.batch_size, self.num_classes], name='target_probs'
            )
            self.max_perturbation = tf.placeholder(tf.float32, self.image_shape, name='max_perturbation')
            self.alpha = tf.placeholder(tf.float32, name='alpha')
            self.learning_rate = tf.placeholder(tf.float32, self.image_shape, name='learning_rate')

            self.main_loss = tf.losses.softmax_cross_entropy(
                self.target_probs,
                self.fake_logits,
                label_smoothing=0.1,
                weights=1.0
            )
            

            self.abs_img_diff = tf.abs(self.fake_image - self.real_image)

            self.reg_loss = self.alpha * tf.reduce_sum(
                tf.where(
                    self.abs_img_diff > self.max_perturbation,
                    self.abs_img_diff, tf.zeros(self.image_shape)
                )
            )

            self.loss = self.main_loss + self.reg_loss

            start_vars = set(x.name for x in tf.global_variables())
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.train = self.optimizer.minimize(self.loss, var_list=[self.fake_image_subst])
            new_vars = [x for x in end_vars if x.name not in start_vars]
            self.init = tf.variables_initializer(var_list=[self.fake_image_subst] + new_vars)

            saver = tf.train.Saver(slim.get_model_variables())
            session_creator = tf.train.ChiefSessionCreator(
                scaffold=tf.train.Scaffold(saver=saver),
                checkpoint_filename_with_path=checkpoint_path,
                master=FLAGS.master
            )
            self.sess = tf.train.MonitoredSession(session_creator=session_creator)
            self.sess.run(self.init)
    
    def inference(self, images, labels, max_perturbation, alpha, learning_rate=10**-3):
        result_images, _, self.sess.run(
            [self.fake_image, self.train],
            feed_dict={
                self.real_image: images,
                self.target_probs: labels,
                self.max_perturbation: max_perturbation,
                self.alpha: alpha,
                self.learning_rate: learning_rate,
            }
        )
        return result_images

In [5]:
pm = PushinMatyshin()

ValueError: Shape must be rank 0 but is rank 4 for 'Adam/update_fake_image_subst/ApplyAdam' (op: 'ApplyAdam') with input shapes: [5,299,299,3], [5,299,299,3], [5,299,299,3], [], [], [5,299,299,3], [], [], [], [5,299,299,3].